In [ ]:
#### LightGBM ####
# 모델 성능 up 모델 학습시간 down
# 반복적인 iteration때 병목 현상이 나타나면 여러 case를 테스트하기 어려운데 그 부분을 해결해줌
# + stacking 으로 모델 성능을 향상시킬 수 있음.

# leaf-wise 방식을 씀 
    # 한쪽으로 치우치기 때문에 overfitting되기 쉬움. 

In [ ]:
#### LightGBM 주요 특징 ####
# 1. Histogram기반 feature binning
    # 연속형 피처들을 특정한 개수의 Bin으로 할당하여 개별 피처들의 범위를 급격히 줄임
    # ==> 수행 시간 줄임
# 2. boosting시 GBDT외에 GOSS, DART 방식 제공
    # GOSS: Gradient값이 큰 값에 대해서만 선택적으로 필터링하여 반복적 재학습
        # : 작으면 이미 진행되었다고 가정
        # : 수행 시간을 줄였지만 모델 성능 향상에는 의미 (gradient가 작은 부분에 대해서는 학습을 하지 않았기 때문)
    # DART: 자잘한 조건을 맞추기 위해 트리가 세분화 되는 것을 방지
# 3. overfitting을 극복하기 위한 hyper parameters

#### LightGBM type ####
# gbdt(default), goss, dart, rt
# ==> 직접 수행해봐야 어떤 것이 가장 좋은지 알 수 있음. 

#### 하이퍼 파라미터 ####
# why? LightGBM의 경우 leaf-wise 방식을 쓰기 때문에 overfitting 되기 쉬움
# 때문에 하이퍼 파라미터 튜닝을 통해 이를 방지하고자 함. 

# 종류:
    # n_estimators
    # learning_rate
    # (과적합을 방지하기 위해 아래 파라미터 사용)
    # max_depth : 트리의 깊이 (=> 보통 피처의 개수가 많으면 max_depth를 10~16으로 둔다고 함.)
    # num_leaves : leaf nodes의 개수
    # subsample : 레코드 개수, 데이터를 샘플링하는 비율 지정
    # colsample_bytree : 피처(컬럼) 개수, 트리 생성에 필요한 피처를 임의로 샘플링
    # min_child_samples : 리프 노드의 최소 데이터 개수
    # reg_lambda : L2 규제
    # reg_alpha : L1 규제 (L1을 조절하는 것이 L2를 조절하는 것보다 성능이 더 좋다는 말이 있음.)

In [ ]:
#### 하이퍼 파라미터 튜닝 방법 ####
# GridSearchCV, Randomized Search
    # 둘은 운빨에 의존함
    # 수행시간이 너무 오래 걸림. 
    # 어느정도 최적화된 하이퍼 파리미터를 활용하여 최적화를 수행하지 않음. 
    
# Bayesian Optimazation (-> 그 이후에 어느 정도의 수동 튜닝.)
# (데이터를 받으면 사후 확률 개선 가능)
    # 점점 많은 값을 받아 수행하면서 사후분포가 점점 개선되고 함수 반환 값을 최대(최소)가 되는
    # 입력 파라미터 영역을 보다 확실하게 찾게 됨.
    
# 하이퍼 파라미터 튜닝
    # 트리구조
        # max_depth
        # num_leaves
        # min_child_samples
        # min_child_weight : 균일도
    # 샘플링 비율
        # subsample
        # colsample_bytree
    # 손실함수 규제
        # reg_lambda
        # reg_alpha
    # feature histogram
        # max_bin
        
# 순서
    # 1. 함수 입력 인자 범위 설정 : baysesian_params = {}
    # 2. 함수 선언
    # 3. 베이지안 객체에 넣어줌
    
# 유의사항 
    # 1. 언제나 최고의 최적화된 파라미터 추출 X
    # 2. 사전에 유일한 파라미터의 범위 제약
    # 3. 가능하면 CV로 함수 결과 반환. But, 최적 하이퍼 파라미터 X
    # 4. 너무 하이퍼 파라미터 튜닝에 의존 X
    # ==> 하아퍼 파라미터 튜닝은 1~2% 좋아지면 좋아진 것! 

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

# parameter 설정 
# parameter는 지속적으로 반복하다보면 범위가 잡힘
    # 수업에서는 선생님께서 자신만의 파라미터를 가지고 있는 분들도 있다고 하심. 
    
bayesian_params = {
    'max_depth': (6, 16), # max_depth는 보통 이 정도로 설정하면 될 것 같음. 
    'num_leaves': (24, 64), 
    'min_child_samples': (10, 200), 
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha': (0.01, 50) 
}

# 함수 선언 : roc_auc_score을 반환해줘야 함. 
def lgb_roc_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree,max_bin, reg_lambda, reg_alpha):
    params = {
        "n_estimators":500, "learning_rate":0.02,
        'max_depth': int(round(max_depth)), #  호출 시 실수형 값이 들어오므로 정수형 하이퍼 파라미터는 정수형으로 변경 
        'num_leaves': int(round(num_leaves)), 
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample': max(min(subsample, 1), 0), 
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'max_bin':  max(int(round(max_bin)),10),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0)
    }
    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                  eval_metric= 'auc', verbose= 100, # verbose는 단지 몇 번마다 결괏값을 출력해줌. 
                  early_stopping_rounds= 100)
    
    valid_proba = lgb_model.predict_proba(valid_x)[:, 1]
    roc_auc = roc_auc_score(valid_y, valid_proba)
    
    return roc_auc  

# iteration 수행 ==> 약 50분이 걸린다고 하심
lgbBO = BayesianOptimization(f = lgb_roc_eval, 
                             pbounds = bayesian_params,
                             random_state=0,)
lgbBO.maximize(init_points=5, n_iter=25)


In [ ]:
# 모든 iteration 반환값을 가지고 있는 .res
lgBO.res 

# dictionary에 있는 target값을 모두 추출
target_list = []
for result in lgbBO.res:
    target = result['target']
    target_list.append(target)
print(target_list)
# 가장 큰 target 값을 가지는 순번(index)를 추출
print('maximum target index:', np.argmax(np.array(target_list)))

# 가장 큰 target값을 가지고 있는 index값을 기준으로 parameter 추출
max_dict = lgbBO.res[np.argmax(np.array(target_list))]
print(max_dict)

In [ ]:
# 위의 파라미터 결괏값을 가지고 모델 학습
def train_apps_all(apps_all_train):
    ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    target_app = apps_all_train['TARGET']

    train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
    print('train shape:', train_x.shape, 'valid shape:', valid_x.shape)
    clf = LGBMClassifier(
                nthread=4,
                n_estimators=1000,
                learning_rate=0.02,
                max_depth = 13,
                num_leaves=57,
                colsample_bytree=0.638,
                subsample=0.682,
                max_bin=435,
                reg_alpha=0.936,
                reg_lambda=4.533,
                min_child_weight=25,
                min_child_samples=166,
                silent=-1,
                verbose=-1,
                )

    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 100, 
            early_stopping_rounds= 100)
    
    return clf

# 이후 예측 및 평가 
preds = clf.predict_proba(apps_all_test.drop('SK_ID_CURR', axis=1))[:, 1 ]
apps_all_test['TARGET'] = preds
apps_all_test[['SK_ID_CURR', 'TARGET']].to_csv('prev_baseline_tuning_01.csv', index=False)

In [ ]:
# 유의사항
# CV가 무조건 좋은 성능을 나타내는 것은 아님 
# 실행해보면서 가장 좋은 것 찾아내기. 